In [2]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.express as px
import torch
from anomalib.models.patchcore.torch_model import PatchcoreModel
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm_notebook as tqdm
import requests
import tarfile

/home/ubuntu/finetuning_cookbook_staging/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



## データセットのダウンロード

- https://www.mvtec.com/company/research/datasets/mvtec-ad/downloads
- https://www.mvtec.com/fileadmin/Redaktion/mvtec.com/company/research/datasets/mvtec_anomaly_detection
- https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420938113-1629952094/mvtec_anomaly_detection.tar.xz
- https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420937370-1629951468/bottle.tar.xz

In [5]:
# ダウンロードするURL
url = "https://www.mydrive.ch/shares/38536/3830184030e49fe74747669442f0f282/download/420937370-1629951468/bottle.tar.xz"
# 保存先ディレクトリ
save_dir = Path("data/mvtec_anomaly_detection")
# 保存するファイル名
file_name = "bottle.tar.xz"
# 保存するファイルのパス
file_path = save_dir / file_name

# ディレクトリが存在しない場合は作成
os.makedirs(save_dir, exist_ok=True)

# ファイルをダウンロードして保存
response = requests.get(url, stream=True)
with open(file_path, 'wb') as file:
    for chunk in response.iter_content(chunk_size=8192):
        file.write(chunk)

# ダウンロードしたファイルを展開
with tarfile.open(file_path, 'r:xz') as tar:
    tar.extractall(path=save_dir)

print(f"ファイルを {save_dir} フォルダにダウンロードして展開しました。")

ファイルを data/mvtec_anomaly_detection フォルダにダウンロードして展開しました。


In [6]:
# モデルの読み込み
model = PatchcoreModel(input_size=(64, 64), layers=["layer1", "layer4"])

# データセットの読み込み
transform = transforms.Compose([transforms.Resize((64, 64)), transforms.ToTensor()])
train_dataset = datasets.ImageFolder(
    root="./data/mvtec_anomaly_detection/bottle/train", transform=transform
)
train_loader = DataLoader(train_dataset, batch_size=len(train_dataset), shuffle=True)
test_dataset = datasets.ImageFolder(
    root="./data/mvtec_anomaly_detection/bottle/test", transform=transform
)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

FeatureExtractor is deprecated. Use TimmFeatureExtractor instead. Both FeatureExtractor and TimmFeatureExtractor will be removed in a future release.


In [7]:

# デバイスの設定（GPUが利用可能な場合）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 学習(教師データをembedding)
model.train()
for data in tqdm(train_loader):
    inputs, _ = data
    inputs = inputs.to(device)
    outputs = model(inputs)
model.subsample_embedding(outputs, 0.1)

RuntimeError: The NVIDIA driver on your system is too old (found version 11020). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver.

In [ ]:

# 推論(テストデータの異常スコアを計算)
model.eval()
results = []
labels = []
images = []
with torch.no_grad():
    for data in tqdm(test_loader):
        inputs, label = data
        images.append(inputs)
        inputs = inputs.to(device)
        result = model(inputs)
        results.append(result)
        labels.append(label)

In [ ]:
# 異常スコアの可視化
# tensor を画像で表示
def convert_to_image(
    image: torch.Tensor,
    max_val: float | None = None,
    min_val: float | None = None,
):
    # tensorをnumpyに変換
    im = image.cpu().detach().numpy()
    # 画像の最大値、最小値を指定
    if max_val is None:
        max_val = im.max()
    if min_val is None:
        min_val = im.min()
    # 画像の正規化
    im = np.transpose(im, (1, 2, 0))
    im = (im - min_val) / (max_val - min_val)
    im = (im * 255).astype(np.uint8)
    # 画像のチャンネル数が1の場合、3チャンネルに変換
    if im.shape[2] == 1:
        im = np.repeat(im, 3, axis=2)
    # 画像の表示
    return im


# ピクセルごとの異常スコアを表示する
i = 8
error_min = results[0][0].min().item()
error_max = results[0][0].max().item()
output_dir = Path("output/anomaly_detection/fig")
os.makedirs(output_dir, exist_ok=True)
for i in range(results[0][0].shape[0]):
    im_error = convert_to_image(results[0][0][i], min_val=error_min, max_val=error_max)
    im_input = convert_to_image(images[0][i])
    px.imshow(im_error).write_image(output_dir / f"{i}_error.png")
    px.imshow(im_input).write_image(output_dir / f"{i}_input.png")

In [ ]:
# 画像異常スコアの分布を確認
# labelsを結合する
all_label = np.concatenate(([l.numpy() for l in labels]))
all_result = np.concatenate([r[1].cpu().detach().numpy() for r in results])
df = pd.DataFrame({"label": all_label, "result": all_result})
# labelカラムをtest_dataset.classesの値に置き換える
df = df.replace({"label": {i: c for i, c in enumerate(test_dataset.classes)}})
fig = px.strip(df, x="label", y="result")
fig.update_layout(
    title="画像異常スコアの分布", xaxis_title="Label", yaxis_title="異常スコア"
)
fig.write_image(output_dir / "violin.png")

In [ ]:
df["is_good_pred"] = df["result"].apply(lambda x: True if x < 10 else False)
df["is_good_true"] = df["label"].apply(lambda x: True if x == "good" else False)
df["correct"] = df["is_good_pred"] == df["is_good_true"]
df["correct"].sum() / len(df)